Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [6]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [7]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.527604
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.710562
Minibatch accuracy: 37.5%
Validation accuracy: 39.4%
Minibatch loss at step 100: 0.760359
Minibatch accuracy: 75.0%
Validation accuracy: 68.0%
Minibatch loss at step 150: 0.655316
Minibatch accuracy: 75.0%
Validation accuracy: 77.3%
Minibatch loss at step 200: 0.417300
Minibatch accuracy: 81.2%
Validation accuracy: 79.0%
Minibatch loss at step 250: 1.559070
Minibatch accuracy: 50.0%
Validation accuracy: 79.2%
Minibatch loss at step 300: 0.673191
Minibatch accuracy: 87.5%
Validation accuracy: 80.3%
Minibatch loss at step 350: 0.520537
Minibatch accuracy: 87.5%
Validation accuracy: 79.1%
Minibatch loss at step 400: 0.475150
Minibatch accuracy: 81.2%
Validation accuracy: 80.7%
Minibatch loss at step 450: 0.338664
Minibatch accuracy: 93.8%
Validation accuracy: 81.4%
Minibatch loss at step 500: 0.470691
Minibatch accuracy: 87.5%
Validation accuracy: 81.9%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [8]:
from math import ceil

batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

def calc_HW(in_size, conv_stride, pool_stride):
    layer1_conv_HW = ceil(in_size / conv_stride)
    layer1_pool_HW = ceil(layer1_conv_HW / pool_stride)
    layer2_conv_HW = ceil(layer1_pool_HW / conv_stride)
    layer2_pool_HW = ceil(layer2_conv_HW / pool_stride)
    return int(layer2_pool_HW)

def init_weight_stddev(fan_in_size):
    return np.sqrt(2.0/float(fan_in_size))

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=init_weight_stddev(patch_size*patch_size*num_channels)))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=init_weight_stddev(patch_size*patch_size*depth)))
  layer2_biases = tf.Variable(tf.zeros([depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [calc_HW(image_size, 2, 2) * calc_HW(image_size, 2, 2) * depth, num_hidden], stddev=init_weight_stddev(calc_HW(image_size, 2, 2) * calc_HW(image_size, 2, 2) * depth)))
  layer3_biases = tf.Variable(tf.zeros([num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=init_weight_stddev(num_hidden)))
  layer4_biases = tf.Variable(tf.zeros([num_labels]))

  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    relu = tf.nn.relu(conv + layer1_biases)
    hidden = tf.nn.max_pool(relu, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    relu = tf.nn.relu(conv + layer2_biases)
    hidden = tf.nn.max_pool(relu, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [9]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.510459
Minibatch accuracy: 6.2%
Validation accuracy: 11.4%
Minibatch loss at step 50: 1.625700
Minibatch accuracy: 50.0%
Validation accuracy: 46.0%
Minibatch loss at step 100: 0.862184
Minibatch accuracy: 75.0%
Validation accuracy: 72.3%
Minibatch loss at step 150: 0.517281
Minibatch accuracy: 87.5%
Validation accuracy: 76.9%
Minibatch loss at step 200: 0.442526
Minibatch accuracy: 87.5%
Validation accuracy: 78.8%
Minibatch loss at step 250: 1.350430
Minibatch accuracy: 56.2%
Validation accuracy: 78.3%
Minibatch loss at step 300: 0.417583
Minibatch accuracy: 81.2%
Validation accuracy: 77.0%
Minibatch loss at step 350: 0.834149
Minibatch accuracy: 68.8%
Validation accuracy: 81.2%
Minibatch loss at step 400: 0.354547
Minibatch accuracy: 93.8%
Validation accuracy: 80.0%
Minibatch loss at step 450: 0.257303
Minibatch accuracy: 100.0%
Validation accuracy: 81.5%
Minibatch loss at step 500: 0.634879
Minibatch accuracy: 75.0%
Validation accuracy: 81.4%
M

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [13]:
from math import ceil, pow

batch_size = 16
num_hidden = 84
patch_size = {'conv1':5, 'pool1':2, 'conv2':5, 'pool2':2, 'conv3':4}
depth = {'conv1':6, 'conv2':16, 'conv3':120}
stride = {'conv1':1, 'pool1':2, 'conv2':1, 'pool2':2, 'conv3':1}
padding = {'conv1':'VALID', 'pool1':'SAME', 'conv2':'VALID', 'pool2':'SAME', 'conv3':'VALID'}

def calc_output_size(in_size, patch, stride, padding):
    if padding == 'VALID':
        return int(ceil((in_size - patch + 1) / stride))
    else:
        return int(ceil(in_size / stride))
    
def calc_output_size_after_conv(in_size, patch, stride, padding):
    layer1_conv_HW = calc_output_size(in_size, patch['conv1'], stride['conv1'], padding['conv1'])
    layer1_pool_HW = calc_output_size(layer1_conv_HW, patch['pool1'], stride['pool1'], padding['pool1'])
    layer2_conv_HW = calc_output_size(layer1_pool_HW, patch['conv2'], stride['conv2'], padding['conv2'])
    layer2_pool_HW = calc_output_size(layer2_conv_HW, patch['pool2'], stride['pool2'], padding['pool2'])
    layer3_conv_HW = calc_output_size(layer2_pool_HW, patch['conv3'], stride['conv3'], padding['conv3'])
    return int(layer3_conv_HW)

def init_weight_stddev(fan_in_size):
    return np.sqrt(2.0/float(fan_in_size))

HW_after_conv = int(pow(calc_output_size_after_conv(image_size,patch_size,stride,padding),2) * depth['conv3'])

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = {'conv1': tf.Variable(tf.truncated_normal(
      [patch_size['conv1'], patch_size['conv1'], num_channels, depth['conv1']], stddev=init_weight_stddev(patch_size['conv1']*patch_size['conv1']*num_channels))),
            'conv2': tf.Variable(tf.truncated_normal(
      [patch_size['conv2'], patch_size['conv2'], depth['conv1'], depth['conv2']], stddev=init_weight_stddev(patch_size['conv2']*patch_size['conv2']*depth['conv1']))),
            'conv3': tf.Variable(tf.truncated_normal(
      [patch_size['conv3'], patch_size['conv3'], depth['conv2'], depth['conv3']], stddev=init_weight_stddev(patch_size['conv3']*patch_size['conv3']*depth['conv2']))),
            'h1': tf.Variable(tf.truncated_normal(
      [HW_after_conv, num_hidden], stddev=init_weight_stddev(HW_after_conv))),
            'h2': tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=init_weight_stddev(num_hidden)))}
  biases = {'conv1': tf.Variable(tf.zeros([depth['conv1']])),
           'conv2': tf.Variable(tf.zeros([depth['conv2']])),
           'conv3': tf.Variable(tf.zeros([depth['conv3']])),
           'h1': tf.Variable(tf.zeros([num_hidden])),
           'h2': tf.Variable(tf.zeros([num_labels]))}

  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, weights['conv1'], [1, stride['conv1'], stride['conv1'], 1], padding=padding['conv1'])
    relu = tf.nn.relu(conv + biases['conv1'])
    hidden = tf.nn.max_pool(relu, [1, patch_size['pool1'], patch_size['pool1'], 1], [1, stride['pool1'], stride['pool1'], 1], padding=padding['pool1'])
    conv = tf.nn.conv2d(hidden, weights['conv2'], [1, stride['conv2'], stride['conv2'], 1], padding=padding['conv2'])
    relu = tf.nn.relu(conv + biases['conv2'])
    hidden = tf.nn.max_pool(relu, [1, patch_size['pool2'], patch_size['pool2'], 1], [1, stride['pool2'], stride['pool2'], 1], padding=padding['pool2'])
    conv = tf.nn.conv2d(hidden, weights['conv3'], [1, stride['conv3'], stride['conv3'], 1], padding=padding['conv3'])
    hidden = tf.nn.relu(conv + biases['conv3'])
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, weights['h1']) + biases['h1'])
    return tf.matmul(hidden, weights['h2']) + biases['h2']
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [14]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.540501
Minibatch accuracy: 12.5%
Validation accuracy: 11.1%
Minibatch loss at step 50: 1.289744
Minibatch accuracy: 68.8%
Validation accuracy: 46.8%
Minibatch loss at step 100: 0.441222
Minibatch accuracy: 87.5%
Validation accuracy: 73.1%
Minibatch loss at step 150: 0.670833
Minibatch accuracy: 81.2%
Validation accuracy: 76.5%
Minibatch loss at step 200: 0.487018
Minibatch accuracy: 93.8%
Validation accuracy: 77.9%
Minibatch loss at step 250: 1.469466
Minibatch accuracy: 50.0%
Validation accuracy: 78.1%
Minibatch loss at step 300: 0.450237
Minibatch accuracy: 87.5%
Validation accuracy: 78.9%
Minibatch loss at step 350: 0.739618
Minibatch accuracy: 81.2%
Validation accuracy: 79.9%
Minibatch loss at step 400: 0.385270
Minibatch accuracy: 87.5%
Validation accuracy: 80.8%
Minibatch loss at step 450: 0.315450
Minibatch accuracy: 93.8%
Validation accuracy: 81.6%
Minibatch loss at step 500: 0.449188
Minibatch accuracy: 93.8%
Validation accuracy: 82.2%
M